In [26]:
import numpy
import gzip
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
import pycountry
import json
import copy

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua
sys.path.append(os.path.expanduser("~")+'/uvpy/')
import uvfunctions as uvf
import uvplot as uvp

import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams['figure.figsize'] = (20, 10)
font = {'family' : 'normal',
    # 'weight' : 'bold',
    'size'   : 22}
matplotlib.rc('font', **font)

from ipywidgets import IntProgress
from IPython.display import display

In [6]:
with open('/users/staff/uvoggenberger/scratch/hug/config_v9/active.json') as json_file:
    active_v9 = json.load(json_file)
with open('/users/staff/uvoggenberger/scratch/hug/config_v11/active.json') as json_file:
    active_v11 = json.load(json_file)

In [7]:
rq = {'variable': ['air_temperature'], 'date': '20060701-20060701', 'format': 'nc'} 
# 0-20999-0-ZEKG 'date': '19901201-19901201', 'statid': ['0-20999-0-ZSAF','0-20999-0-TTCC'], 'optional': ['wind_bias_estimate',], 'plev':70000, 

In [8]:
df_v9 = eua.vm_request_wrapper(rq , overwrite=True, vm_url='http://127.0.0.1:8009')
len(df_v9)

719

In [9]:
df_v11 = eua.vm_request_wrapper(rq , overwrite=True)#, vm_url='http://127.0.0.1:8009')
len(df_v11)

741

In [19]:
testlist = list(df_v9.keys())
out = []
for i in df_v11:
    if ~np.isin(i, testlist):
        out.append(i)
        # display(df_v11[i])
        # display(df_v11[i].to_dataframe())
display(out)
print(len(out))

['dest_0-20000-0-04018_air_temperature',
 'dest_0-20000-0-12982_air_temperature',
 'dest_0-20000-0-17130_air_temperature',
 'dest_0-20000-0-26072_air_temperature',
 'dest_0-20000-0-64650_air_temperature',
 'dest_0-20000-0-65418_air_temperature',
 'dest_0-20000-0-82193_air_temperature',
 'dest_0-20000-0-82917_air_temperature',
 'dest_0-20001-0-64910_air_temperature',
 'dest_0-20600-0-53068_air_temperature',
 'dest_0-20600-0-53772_air_temperature',
 'dest_0-20600-0-69011_air_temperature',
 'dest_0-20999-0-01004_air_temperature',
 'dest_0-20999-0-20744_air_temperature',
 'dest_0-20999-0-ASUK1_air_temperature',
 'dest_0-20999-0-FQFM_air_temperature',
 'dest_0-20999-0-JCCX_air_temperature',
 'dest_0-20999-0-JGQH_air_temperature',
 'dest_0-20999-0-LDWR_air_temperature',
 'dest_0-20999-0-OXGN2_air_temperature',
 'dest_0-246-0-101104_air_temperature',
 'dest_0-246-0-101339_air_temperature',
 'dest_0-246-0-101932_air_temperature',
 'dest_0-620-2001-08522_air_temperature',
 'dest_20999-coordinat

31


In [41]:
missing = [('04018', 63.97, 337.4), ('12982', 46.25, 20.1), ('13275', 44.78, 20.53), ('17130', 39.95, 32.88), ('47744', 35.43, 133.35), ('51886', 38.25, 90.85), ('52652', 38.93, 100.43), ('55664', 28.58, 86.62), ('56173', 31.85, 102.7), ('61995', -20.3, 57.5), ('63705', 0.05, 32.45), ('67964', -20.15, 28.62), ('74547', 38.62, 262.7), ('74646', 36.62, 262.52), ('74650', 35.68, 264.13), ('94302', -22.23, 114.08), ('DBBH', 43.5, 310.0), ('ENQT', 37.2, 350.9), ('FNOR', 25.5, 308.0), ('FNRS', 42.8, 344.7), ('JBOA', 31.0, 127.9), ('JCCX', 32.4, 128.1), ('JDWX', 41.2, 141.9), ('JGQH', 18.5, 136.9), ('JIVB', 34.8, 130.0), ('LDWR', 66.0, 2.1), ('OXYH2', 60.2, 352.6), ('UCKZ', -69.4, 74.6)]
display(missing)
len(missing)

[('04018', 63.97, 337.4),
 ('12982', 46.25, 20.1),
 ('13275', 44.78, 20.53),
 ('17130', 39.95, 32.88),
 ('47744', 35.43, 133.35),
 ('51886', 38.25, 90.85),
 ('52652', 38.93, 100.43),
 ('55664', 28.58, 86.62),
 ('56173', 31.85, 102.7),
 ('61995', -20.3, 57.5),
 ('63705', 0.05, 32.45),
 ('67964', -20.15, 28.62),
 ('74547', 38.62, 262.7),
 ('74646', 36.62, 262.52),
 ('74650', 35.68, 264.13),
 ('94302', -22.23, 114.08),
 ('DBBH', 43.5, 310.0),
 ('ENQT', 37.2, 350.9),
 ('FNOR', 25.5, 308.0),
 ('FNRS', 42.8, 344.7),
 ('JBOA', 31.0, 127.9),
 ('JCCX', 32.4, 128.1),
 ('JDWX', 41.2, 141.9),
 ('JGQH', 18.5, 136.9),
 ('JIVB', 34.8, 130.0),
 ('LDWR', 66.0, 2.1),
 ('OXYH2', 60.2, 352.6),
 ('UCKZ', -69.4, 74.6)]

28

In [20]:
# check by name:
not_in_diff = []
for i in missing:
    if np.invert(i[0] in '\t'.join(out)):
        not_in_diff.append(i)
        print(i)
print(len(not_in_diff))

('13275', 44.78, 20.53)
('47744', 35.43, 133.35)
('51886', 38.25, 90.85)
('52652', 38.93, 100.43)
('55664', 28.58, 86.62)
('56173', 31.85, 102.7)
('61995', -20.3, 57.5)
('63705', 0.05, 32.45)
('67964', -20.15, 28.62)
('74547', 38.62, 262.7)
('74646', 36.62, 262.52)
('74650', 35.68, 264.13)
('94302', -22.23, 114.08)
('DBBH', 43.5, 310.0)
('ENQT', 37.2, 350.9)
('FNOR', 25.5, 308.0)
('FNRS', 42.8, 344.7)
('JBOA', 31.0, 127.9)
('JDWX', 41.2, 141.9)
('JIVB', 34.8, 130.0)
('OXYH2', 60.2, 352.6)
('UCKZ', -69.4, 74.6)
22


In [39]:
# check by lat/lon:
remaining = copy.deepcopy(not_in_diff)
found = []
found_where = []
max_count = len(out)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

for i in out:
    df_i = df_v11[i].to_dataframe()
    for j in not_in_diff:
        if np.logical_and(((df_i.lat.iloc[0] - j[1]) < 0.3), ((df_i.lon.iloc[0] - j[2]) < 0.3)):
            found.append(j)
            found_where.append(i)
            try:
                remaining.remove(j)
            except:
                pass
    f.value += 1
display(remaining)
print(len(remaining))

IntProgress(value=0, max=31)

[('61995', -20.3, 57.5),
 ('67964', -20.15, 28.62),
 ('94302', -22.23, 114.08),
 ('UCKZ', -69.4, 74.6)]

4


In [ ]:
# ^ those 4 missing stations could not be found in the "new additions" from v11.